In [253]:
import pandas as pd
import datetime
import time
import numpy as np
from sqlalchemy import create_engine
import pyodbc
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import warnings
from statsmodels.tsa.seasonal import seasonal_decompose 
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller,pacf,acf
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from prophet import Prophet
import autots
from auto_ts import auto_timeseries
from sklearn.preprocessing import StandardScaler
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.holtwinters import SimpleExpSmoothing
import pyod.models.copod
from pyod.models.copod import COPOD
#from catboost import CatBoostRegressor
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, SimpleRNN

In [254]:
connStr = pyodbc.connect("""DRIVER={SQL Server};
SERVER=test-sql-05;
DATABASE=GENERAL_ACCESS;
Trusred_Connection=yes""")
cursor= connStr.cursor()

OperationalError: ('08001', '[08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]SQL Server не существует, или доступ запрещен. (17) (SQLDriverConnect); [08001] [Microsoft][ODBC SQL Server Driver][DBNETLIB]ConnectionOpen (Connect()). (53); [08001] [Microsoft][ODBC SQL Server Driver]Недопустимый атрибут строки соединения (0)')

In [ ]:
query = "SELECT * FROM dbo.CC_IncomeCalls"
income_calls = pd.read_sql(query, connStr)
income_calls.tail()

In [ ]:
query = "SELECT * FROM dbo.CC_OnlineOrders"
orders = pd.read_sql(query, connStr)
orders = orders.rename(columns = {'ORDER_DATE':'call_dt','QTY':'order_qty'})
orders = orders.drop(orders.tail(7).index)
orders.tail()

In [ ]:
query = "SELECT * FROM dbo.CC_MissingCalls"
missing_calls = pd.read_sql(query, connStr)
missing_calls.head()

In [ ]:
query = "SELECT * FROM dbo.CC_OutcomeCalls"
outcome_calls = pd.read_sql(query, connStr)
outcome_calls.head()

In [ ]:
query = "SELECT * FROM [GENERAL_ACCESS].[dbo].[СС_Discounts]"
discounts = pd.read_sql(query, connStr)
discounts.head()

In [ ]:
query = "SELECT * FROM [GENERAL_ACCESS].[dbo].[СС_Sales]"
sales1 = pd.read_sql(query, connStr)
sales1 = sales1.rename(columns = {'SALE_DATE':'call_dt','CHQ_QTY':'chq_qty'})
sales1.head()

In [ ]:
query = "SELECT * FROM dbo.СС_Web_visits"
web_visits = pd.read_sql(query, connStr)
web_visits.head()

In [ ]:
# перевод дат в нужный формат
missing_calls['call_dt']=pd.to_datetime(missing_calls['call_dt'], format='%Y-%m-%d')
income_calls['call_dt']=pd.to_datetime(income_calls['call_dt'], format='%Y-%m-%d')
outcome_calls['call_dt']=pd.to_datetime(outcome_calls['call_dt'], format='%Y-%m-%d')
discounts['disc_dt']=pd.to_datetime(discounts['disc_dt'], format='%Y-%m-%d')
sales1['call_dt']=pd.to_datetime(sales1['call_dt'], format='%Y-%m-%d')
web_visits['visit_dt']=pd.to_datetime(web_visits['visit_dt'], format='%Y-%m-%d')
orders['call_dt']=pd.to_datetime(orders['call_dt'], format='%Y-%m-%d')

In [ ]:
# датафрейм по часам
income_calls = income_calls.drop(['operator','phone_number'], axis =1)
outcome_calls = outcome_calls.drop(['operator','phone_number'], axis =1)
missing_calls = missing_calls.drop(['operator','phone_number'], axis =1)
calls = pd.concat([income_calls,outcome_calls,missing_calls]).reset_index(drop=True)
calls['year']  = calls['call_dt'].dt.year
calls['weekday']  = calls['call_dt'].dt.dayofweek
calls['month']  = calls['call_dt'].dt.month
calls['hour'] = pd.DatetimeIndex(calls['call_time']).hour
calls['hour'] = (calls['hour'].astype(int) + 1).astype(float)
calls = calls.assign(number=1)
calls = calls.sort_values(['call_dt', 'call_time']).reset_index(drop=True)
calls.tail(10)

In [ ]:
#датафрейм по дням
inc = income_calls.groupby('call_dt')['id_call'].count().to_frame()
out = outcome_calls.groupby('call_dt')['id_call'].count().to_frame()
miss = missing_calls.groupby('call_dt')['id_call'].count().to_frame()
all_calls = pd.merge(inc, out, on = 'call_dt', how = 'outer')
all_calls = pd.merge(all_calls, miss, on = 'call_dt', how = 'outer')
all_calls = pd.merge(all_calls, orders, on = 'call_dt', how = 'outer')
discounts = discounts.rename(columns = {'disc_dt':'call_dt'})
all_calls = pd.merge(all_calls, discounts, on = 'call_dt', how = 'outer')
all_calls = all_calls.rename(columns = {'id_call_x':'income_calls','id_call_y':'outcome_calls', 'id_call':'missed_calls'})
web_visits = web_visits.rename(columns = {'visit_dt':'call_dt'})
all_calls = pd.merge(all_calls, web_visits, on = 'call_dt', how = 'outer')
all_calls['sum_calls'] = all_calls['income_calls'].fillna(0)+all_calls['outcome_calls'].fillna(0)+all_calls['missed_calls'].fillna(0)
all_calls [['income_calls','outcome_calls','missed_calls']] = all_calls [['income_calls','outcome_calls','missed_calls']].fillna(0)
all_calls ['disc_type'].fillna('Нет', inplace = True)
sales = sales1.groupby('call_dt')['chq_qty'].sum().to_frame()
all_calls = pd.merge(all_calls, sales, on = 'call_dt', how = 'outer')
all_calls['weekday']  = all_calls['call_dt'].dt.dayofweek
all_calls['month']  = all_calls['call_dt'].dt.month
all_calls['year']  = all_calls['call_dt'].dt.year

for i, row in all_calls.iterrows():
    if row['disc_type'] == 'Купон':
        all_calls.at [i, 'discCOUPON'] = True
    elif row['disc_type'] == '15 число':
        all_calls.at [i, 'disc15'] = True
    elif row['disc_type'] == 'ШОТ':
        all_calls.at [i, 'discSHOT'] = True
    else: pass

all_calls[['discCOUPON', 'disc15', 'discSHOT']] = all_calls[['discCOUPON', 'disc15', 'discSHOT']].fillna(False)
all_calls = all_calls.sort_values(['call_dt']).reset_index(drop=True)
all_calls.drop(all_calls.tail(1).index,inplace=True)
all_calls 

In [ ]:
status_2021 = calls[calls['year'] == 2021].groupby('call_status')['id_call'].count()
status_2022 = calls[calls['year'] == 2022].groupby('call_status')['id_call'].count()
status_2023 = calls[calls['year'] == 2023].groupby('call_status')['id_call'].count()

status_calls = pd.concat([status_2021, status_2022, status_2023], axis=1).fillna(0)
status_calls.columns = ['2021', '2022', '2023']
pd.set_option('display.max_rows', None)
status_calls = status_calls.reset_index()
status_calls.columns = ['type', '2021', '2022', '2023']
status_calls

In [ ]:
fig, ax = plt.subplots(figsize=(20, 20))


ax.barh(status_2021.index, status_2021.values, align='center', label='2021')
ax.barh(status_2022.index, status_2022.values, align='center', label='2022')
ax.barh(status_2023.index, status_2023.values, align='center', label='2023')


ax.set_xlabel('Количество звонков')
ax.set_ylabel('Статус звонка')
ax.set_title('Столбчатая диаграмма по статусу звонков каждый год')

ax.legend()

plt.show()

In [ ]:
region_calls = pd.concat([region_2021, region_2022, region_2023], axis=1).fillna(0)
region_calls.columns = ['2021', '2022', '2023']
pd.set_option('display.max_rows', None)
region_calls = region_calls.reset_index()
region_calls.columns = ['type', '2021', '2022', '2023']
region_calls

In [ ]:
region_2021 = calls[calls['year'] == 2021].groupby('region')['id_call'].count()
region_2022 = calls[calls['year'] == 2022].groupby('region')['id_call'].count()
region_2023 = calls[calls['year'] == 2023].groupby('region')['id_call'].count()

fig, ax = plt.subplots(figsize=(50, 30))

ax.barh(region_2021.index, region_2021.values, label='2021')
ax.barh(region_2022.index, region_2022.values, label='2022')
ax.barh(region_2023.index, region_2023.values, label='2023')


ax.set_xlabel('Количество звонков')
ax.set_ylabel('Регион звонка')
ax.set_title('Столбчатая диаграмма по регионам звонков каждый год')

ax.legend()

plt.show()

In [ ]:
types_to_calls = {'Адреса маг-ов/ Общая инф-ия по сети':'Информация по сети',
       'Карта лояльности':'Выгода с Улыбкой',
       'Интернет-магазин':'Интернет-магазин',
       'Статус заказа':'Заказ', 
       'Возврат/обмен':'Заказ',
       'Наличие товара':'Информация о товаре', 
       'Подарочный сертификат':'Выгода с Улыбкой', 
       'Вакансия':'Вакансия', 
       'Акции':'Выгода с Улыбкой',
       'Сотрудник':'Сотрудники', 
       'Другое':'Другое', 
       'Общая информация':'Информация по сети', 
       'Жалоба':'Жалоба',
       'Перевод на сотрудника ИМ':'Сотрудники', 
       'Звонок не состоялся':'Другое',
       'Оставлен в магазине':'Другое', 
       'Наличие в магазине/ИМ':'Информация о товаре', 
       'Консультация':'Информация по сети',
       'Коммерческое предложение':'Другое', 
       'Информация по доставке':'Заказ',
       'Проблема в МП':'Интернет-магазин', 
       'Обучение нового сотрудника':'Сотрудники', 
       'Сделать заказ':'Заказ',
       'Онлайн-оплата':'Интернет-магазин', 
       'Проблема на сайте':'Интернет-магазин', 
       'Информация по заказу':'Заказ',
       'Хулиганы':'Другое', 
       'Отдел продаж':'Информация по сети', 
       'Дата доставки':'Заказ', 
       'Тишина':'Другое',
       'Сервис СберМаркет':'Заказы со сторонних сервисов', 
       'Отменить заказ':'Заказ', 
       'Отмена заказа':'Заказ',
       'Проверка ОПП':'Информация по сети', 
       'Благодарность':'Другое',
       'Технический сбой в работе внутренних программ':'Жалоба',
       'Статус "Сотрудник" Изменение данных':'Сотрудники', 
       'Кризис 2022':'Другое',
       'Звонок без консультации':'Другое', 
       'Контакты с сотрудниками компании':'Сотрудники',
       'Заказы PickupinStore':'Заказ', 
       'Консультация по заказу':'Заказ',
       'Сотрудник Delivery':'Заказы со сторонних сервисов', 
       'Заказ AliExpress':'Заказы со сторонних сервисов',
       'Оформление заказа Сайт':'Интернет-магазин',
       'Проблема на Сайте':'Интернет-магазин', 
       'Новая версия МП':'Интернет-магазин', 
       'Перевод на сотрудника КЦ':'Сотрудники',
       'Электронные чеки':'Другое',     
       'Адреса маг-ов/ Режим работы / Общая инф-ия по сети':'Информация по сети',
       'Жалоба на сотрудника':'Жалоба', 
       'Информация о товаре':'Информация о товаре',
       'Оформление заказа в МП':'Заказ', 
       'Консультация по ИМ':'Интернет-магазин', 
       'Заказ ProSkidku':'Заказы со сторонних сервисов',
       'Заказ  Delivery':'Заказы со сторонних сервисов', 
       'Заказ СберМегаМаркет':'Заказы со сторонних сервисов',
       'МОБИЛИЗАЦИЯ':'Другое',
       'СОЦ.СЕТИ / ВЗЛОМ':'Другое', 
       'Заказ Яндекс.Еда':'Заказы со сторонних сервисов', 
       'Жалоба/проблема':'Жалоба',
       'Заказ OZON':'Заказы со сторонних сервисов', 
       'Кризис 2022 – Lamel':'Другое'}

In [ ]:
calls['category'] = calls['call_status']
calls['category'] = calls['category'].replace(types_to_calls)
calls['category'] = calls['category'].fillna('Нет категории')
calls.tail(10)

In [ ]:
calls['category'].unique()

In [ ]:
calls['duration_time_sec']=\
calls.duration_time.fillna('00:00:00').apply(lambda x:datetime.timedelta(hours= time.strptime(x,'%H:%M:%S').tm_hour,
                                                       minutes= time.strptime(x,'%H:%M:%S').
                                                       tm_min,seconds=time.strptime(x,'%H:%M:%S').tm_sec).total_seconds())
calls.tail(10)

In [ ]:
calls.loc[calls['category'] == 'Нет категории', 'category'] = calls['duration_time_sec'].apply(
    lambda x: 'Другое' if x > 0 else 'Звонок не состоялся'
)
calls.loc[calls['id_call'].str.startswith('out-'), 'category'] = 'Исходящий звонок'


calls.tail(10)

In [ ]:
# топ категорий по количеству
np.round(((calls.groupby('category')['id_call'].count())/(calls['id_call'].count())).sort_values(ascending=False),2) 

In [ ]:
# топ категорий по количеству 2023
np.round(((calls[calls['year'] == 2023].groupby('category')['id_call'].count())/(calls[calls['year'] == 2023]['id_call'].count())).sort_values(ascending=False), 2)

In [ ]:
# топ категорий по длительности
calls.groupby('category')['duration_time_sec'].mean().sort_values(ascending=False)

In [ ]:
# топ категорий по длительности 2023
calls[calls['year'] == 2023].groupby('category')['duration_time_sec'].mean().sort_values(ascending=False)

In [ ]:
df_count = calls.groupby(['call_dt', 'category']).agg({'category': 'count'}).rename(columns={'category': 'num_calls'}).reset_index()
cal = pd.merge(df_count, all_calls[['call_dt','order_qty','qty','chq_qty','discSHOT','disc15','discCOUPON','weekday','month','year']], on = 'call_dt', how = 'outer')
cal = cal.drop(cal.tail(20).index)
cal['actiya'] = cal[['discSHOT', 'disc15', 'discCOUPON']].any(axis=1)
cal.sort_values(['call_dt']).tail(10)

In [ ]:
plt.figure(figsize=(50,10))
plt.plot(all_calls['call_dt'], all_calls['income_calls'], label = 'входящие')

In [ ]:
plt.figure(figsize=(50,10))
plt.plot(all_calls['call_dt'], all_calls['outcome_calls'], label = 'исходящие') #почему-то стали больше звонить из колл-центра

In [ ]:
df = calls[calls['category'] == 'Исходящий звонок']
df = df.sort_values(['duration_time_sec']).reset_index(drop=True)
df.head(20)

In [ ]:
plt.figure(figsize=(50,15))
plt.hist(df['duration_time_sec'], bins=1000, edgecolor='black')
plt.xlim(0, 300)

In [ ]:
plt.figure(figsize=(50,10))
plt.plot(all_calls['call_dt'], all_calls['missed_calls'], label = 'пропущенные')

In [ ]:
filtered_data = cal[cal['category'] == 'Заказ'] 

plt.figure(figsize=(50,10))
plt.plot(filtered_data['call_dt'], filtered_data['num_calls'], label = 'звонки')

In [ ]:
filtered_data = cal[cal['category'] == 'Заказы со сторонних сервисов']  

plt.figure(figsize=(50,10))
plt.plot(filtered_data['call_dt'], filtered_data['num_calls'], label = 'звонки')

In [ ]:
filtered_data = cal[cal['category'] == 'Выгода с Улыбкой'] 

plt.figure(figsize=(50,10)) 
filtered_data['disc15'] = filtered_data.apply(lambda row: row['num_calls'] if row['disc15'] else 0, axis=1)
filtered_data['discSHOT'] = filtered_data.apply(lambda row: row['num_calls'] if row['discSHOT'] else 0, axis=1)
filtered_data['discCOUPON'] = filtered_data.apply(lambda row: row['num_calls'] if row['discCOUPON'] else 0, axis=1)
plt.plot(filtered_data['call_dt'], filtered_data['num_calls'], label = 'звонки')
plt.plot(filtered_data['call_dt'], filtered_data['disc15'], marker = 'o', linestyle = '', color = 'red')
plt.plot(filtered_data['call_dt'], filtered_data['discSHOT'], marker = 'o', linestyle = '', color = 'orange')
plt.plot(filtered_data['call_dt'], filtered_data['discCOUPON'], marker = 'o', linestyle = '', color = 'green')


In [ ]:
filtered_data = cal[cal['category'] == 'Интернет-магазин']  

plt.figure(figsize=(50,10))
plt.plot(filtered_data['call_dt'], filtered_data['num_calls'], label = 'звонки')

In [ ]:
filtered_data = cal[cal['category'] == 'Информация по сети'] 

plt.figure(figsize=(50,10))
plt.plot(filtered_data['call_dt'], filtered_data['num_calls'], label = 'звонки')

In [ ]:
filtered_data = cal[cal['category'] == 'Информация о товаре']  

plt.figure(figsize=(50,10))
plt.plot(filtered_data['call_dt'], filtered_data['num_calls'], label = 'звонки')

In [ ]:
filtered_data = cal[cal['category'] == 'Вакансия'] 

plt.figure(figsize=(50,10))
plt.plot(filtered_data['call_dt'], filtered_data['num_calls'], label = 'звонки')

In [ ]:
filtered_data = cal[cal['category'] == 'Сотрудники'] 

plt.figure(figsize=(50,10))
plt.plot(filtered_data['call_dt'], filtered_data['num_calls'], label = 'звонки')

In [ ]:
filtered_data = cal[cal['category'] == 'Жалоба']  

plt.figure(figsize=(50,10))
plt.plot(filtered_data['call_dt'], filtered_data['num_calls'], label = 'звонки')

In [ ]:
filtered_data = cal[cal['category'] == 'Другое']  

plt.figure(figsize=(50,10))
plt.plot(filtered_data['call_dt'], filtered_data['num_calls'], label = 'звонки')

In [ ]:
filtered_data = cal[cal['category'] == 'Исходящий звонок'] 

plt.figure(figsize=(50,10))
plt.plot(filtered_data['call_dt'], filtered_data['num_calls'], label = 'звонки')

In [ ]:
filtered_data = cal[cal['category'] == 'Звонок не состоялся'] 

plt.figure(figsize=(50,10))
plt.plot(filtered_data['call_dt'], filtered_data['num_calls'], label = 'звонки')

## Корреляция и сезоность

In [ ]:
correlation_matrix = all_calls[['sum_calls', 'qty', 'chq_qty', 'discSHOT', 'disc15', 'discCOUPON', 'order_qty']].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')

In [ ]:
categories = cal['category'].unique()
for category in categories:
    subset = cal[cal['category'] == category]
    correlation = subset[['num_calls', 'order_qty', 'qty', 'chq_qty', 'actiya']].corr()
    print(category)
    print(correlation)
    print()

In [ ]:
y21 = all_calls[all_calls['year'] == 2022]
y21['disc15'] = y21.apply(lambda row: row['sum_calls'] if row['disc15'] else 0, axis=1)
y21['discSHOT'] = y21.apply(lambda row: row['sum_calls'] if row['discSHOT'] else 0, axis=1)
y21['discCOUPON'] = y21.apply(lambda row: row['sum_calls'] if row['discCOUPON'] else 0, axis=1)

fig, ax = plt.subplots(figsize=(40, 10))
plt.plot(y21['call_dt'], y21['sum_calls'], label = 'звонки')
plt.plot(y21['call_dt'], y21['disc15'], marker = 'o', linestyle = '', color = 'red')
plt.plot(y21['call_dt'], y21['discSHOT'], marker = 'o', linestyle = '', color = 'orange')
plt.plot(y21['call_dt'], y21['discCOUPON'], marker = 'o', linestyle = '', color = 'green')


In [ ]:
week1 = all_calls[all_calls['year'] == 2021].groupby('weekday')['sum_calls'].mean().to_frame() 
week2 = all_calls[all_calls['year'] == 2022].groupby('weekday')['sum_calls'].mean().to_frame() 
week3 = all_calls[all_calls['year'] == 2023].groupby('weekday')['sum_calls'].mean().to_frame() 
plt.plot(week1, color = 'red')
plt.plot(week2, color = 'green')
plt.plot(week3, color = 'blue')
plt.show() # 0 - пн, 6 - вс. Они работают с 9 до 9 без вых, -> люди предпочитают звонить в начале недели

In [ ]:
hour1 = calls[calls['year'] == 2021].groupby('hour')['number'].count().to_frame().astype(float)
hour2 = calls[calls['year'] == 2022].groupby('hour')['number'].count().to_frame().astype(float)
hour3 = calls[calls['year'] == 2023].groupby('hour')['number'].count().to_frame().astype(float)
plt.plot(hour1)
plt.plot(hour2)
plt.plot(hour3)
plt.show() # они работают с 9 до 9  -> люди предпочитают звонить в середине дня

In [ ]:
month1 = all_calls[all_calls['year'] == 2021].groupby('month')['sum_calls'].mean().to_frame()
month2 = all_calls[all_calls['year'] == 2022].groupby('month')['sum_calls'].mean().to_frame()
month3 = all_calls[all_calls['year'] == 2023].groupby('month')['sum_calls'].mean().to_frame()
plt.plot(month1, color = 'blue', label='2021')
plt.plot(month2, color = 'crimson', label='2022')
plt.plot(month3, color = 'green', label='2023')
plt.xlabel('Месяц')
plt.ylabel('Количество звонков')
plt.title('Динамика звонков по годам')
plt.legend()
plt.show()

In [ ]:
# X-В
decompose_result = seasonal_decompose(all_calls['sum_calls'], period =30)
decompose_result.plot()

In [ ]:
# Д-Ф на стац
result = adfuller(all_calls['sum_calls'])
result

## Прогноз "Жалобa"

In [ ]:
start_date = pd.to_datetime('2023-05-01')
filtered_d = cal[cal['category'] == 'Жалоба']
filtered_d1 = filtered_d[filtered_d['call_dt'] < start_date ]
filtered_d1

In [ ]:
model = COPOD(contamination=0.05)
filtered_data1 = filtered_d1
model.fit(filtered_data1['num_calls'].values.reshape(-1, 1))
outliers = model.predict(filtered_data1['num_calls'].values.reshape(-1, 1))
count = np.count_nonzero(outliers == 1)     
filtered_data1['outliers'] = outliers
filtered_data1 = filtered_data1.reset_index(drop=True)

outl = filtered_data1.loc[filtered_data1['outliers'] == 1]
outl

In [ ]:
plt.figure(figsize=(50,10))
plt.plot(filtered_data1['call_dt'], filtered_data1['num_calls'], label = 'звонки')
plt.plot(outl['call_dt'], outl['num_calls'], marker = 'o', linestyle = '', label = 'звонки')

In [ ]:
# Замена значений выбросов в столбце calls
for i in range(len(filtered_data1)):
    if filtered_data1.at[i, 'outliers'] == 1:
        prev_mean = filtered_data1.at[max(i-2, 0), 'num_calls']
        filtered_data1.at[i, 'num_calls'] = (prev_mean) 

filtered_d1 = filtered_data1       
plt.figure(figsize=(50,10))
plt.plot(filtered_data1['call_dt'], filtered_data1['num_calls'], label = 'звонки')

In [ ]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing, SimpleExpSmoothing, Holt

In [ ]:
result = adfuller(filtered_d1['num_calls'])
result

In [ ]:
model = SimpleExpSmoothing(filtered_d1['num_calls'])
forecast_complaints = pd.DataFrame(columns=["call_dt", "forc"], dtype=float)
forecast_complaints["call_dt"] = pd.to_datetime(forecast_complaints["call_dt"], format='%Y-%m-%d')
forecast_complaints["forc"] = forecast_complaints["forc"].astype(float)
forecast = model.fit().forecast(steps=120)
for i, value in enumerate(forecast):
    date = start_date + pd.DateOffset(days=i)
    forecast_complaints.loc[i] = [date, value]
    
forecast_complaints

In [ ]:
plt.figure(figsize=(16, 6))
plt.plot(filtered_d['call_dt'], filtered_d['num_calls'], label='Actual')
plt.plot(forecast_complaints['call_dt'], forecast_complaints['forc'], label='Forecast')
plt.xlabel('Дата')
plt.ylabel('Количество звонков')
plt.title('Прогноз по категории "Жалобы"')
plt.legend()
plt.show()

## Прогноз "Заказы сторонних сервисов"

In [ ]:
start_date = pd.to_datetime('2023-05-01')
beg_date = pd.to_datetime('2022-09-30')
filtered_d = cal[cal['category'] == 'Заказы со сторонних сервисов']
filtered_d1 = filtered_d[(filtered_d['call_dt'] < start_date) & (filtered_d['call_dt'] > beg_date)]
filtered_d1

In [ ]:
model = COPOD(contamination=0.05)
filtered_data1 = filtered_d1
model.fit(filtered_data1['num_calls'].values.reshape(-1, 1))
outliers = model.predict(filtered_data1['num_calls'].values.reshape(-1, 1))
count = np.count_nonzero(outliers == 1)     
filtered_data1['outliers'] = outliers
filtered_data1 = filtered_data1.reset_index(drop=True)

outl = filtered_data1.loc[filtered_data1['outliers'] == 1]
outl

In [ ]:
plt.figure(figsize=(50,10))
plt.plot(filtered_data1['call_dt'], filtered_data1['num_calls'], label = 'звонки')
plt.plot(outl['call_dt'], outl['num_calls'], marker = 'o', linestyle = '', label = 'звонки')


In [ ]:
# Замена значений выбросов в столбце calls
for i in range(len(filtered_data1)):
    if filtered_data1.at[i, 'outliers'] == 1:
        prev_mean = filtered_data1.at[max(i-2, 0), 'num_calls']
        filtered_data1.at[i, 'num_calls'] = (prev_mean) 

filtered_d1 = filtered_data1       
plt.figure(figsize=(50,10))
plt.plot(filtered_data1['call_dt'], filtered_data1['num_calls'], label = 'звонки')

In [ ]:
result = adfuller(filtered_d1['num_calls'])
result

In [ ]:
model = SimpleExpSmoothing(filtered_d1['num_calls'])
forecast_marketplaces = pd.DataFrame(columns=["call_dt", "forc"], dtype=float)
forecast = model.fit().forecast(steps=120)
for i, value in enumerate(forecast):
    date = start_date + pd.DateOffset(days=i)
    forecast_marketplaces.loc[i] = [date, value]
forecast_marketplaces

In [ ]:
plt.figure(figsize=(16, 6))
plt.plot(filtered_d['call_dt'], filtered_d['num_calls'], label='Actual')
plt.plot(forecast_marketplaces['call_dt'], forecast_marketplaces['forc'], label='Forecast')
plt.xlabel('Дата')
plt.ylabel('Количество звонков')
plt.title('Прогноз по категории "Заказы сторонних серверов"')
plt.legend()
plt.show()

## Прогноз "Заказ"

In [ ]:
start_date = pd.to_datetime('2023-05-01')
beg_date = pd.to_datetime('2022-06-01')
filtered_d = cal[cal['category'] == 'Заказ']
filtered_d1 = filtered_d[(filtered_d['call_dt'] < start_date) & (filtered_d['call_dt'] > beg_date)]
filtered_d1

In [ ]:
model = COPOD(contamination=0.05)
filtered_data1 = filtered_d1
model.fit(filtered_data1['num_calls'].values.reshape(-1, 1))
outliers = model.predict(filtered_data1['num_calls'].values.reshape(-1, 1))
count = np.count_nonzero(outliers == 1)     
filtered_data1['outliers'] = outliers
filtered_data1 = filtered_data1.reset_index(drop=True)

outl = filtered_data1.loc[filtered_data1['outliers'] == 1]
outl

In [ ]:
plt.figure(figsize=(50,10))
plt.plot(filtered_data1['call_dt'], filtered_data1['num_calls'], label = 'звонки')
plt.plot(outl['call_dt'], outl['num_calls'], marker = 'o', linestyle = '', label = 'звонки')

In [ ]:
# Замена значений выбросов в столбце calls
for i in range(len(filtered_data1)):
    if filtered_data1.at[i, 'outliers'] == 1:
        prev_mean = filtered_data1.at[max(i-2, 0), 'num_calls']
        filtered_data1.at[i, 'num_calls'] = (prev_mean) 

filtered_d1 = filtered_data1       
plt.figure(figsize=(50,10))
plt.plot(filtered_data1['call_dt'], filtered_data1['num_calls'], label = 'звонки')

In [ ]:
result = adfuller(filtered_d1['num_calls'])
result

In [ ]:
model = SimpleExpSmoothing(filtered_d1['num_calls'])
forecast_order = pd.DataFrame(columns=["call_dt", "forc"], dtype=float)
forecast = model.fit().forecast(steps=120)
for i, value in enumerate(forecast):
    date = start_date + pd.DateOffset(days=i)
    forecast_order.loc[i] = [date, value]
forecast_order

In [ ]:
plt.figure(figsize=(16, 6))
plt.plot(filtered_d['call_dt'], filtered_d['num_calls'], label='Actual')
plt.plot(forecast_order['call_dt'], forecast_order['forc'], label='Forecast')
plt.xlabel('Дата')
plt.ylabel('Количество заказов')
plt.title('Прогноз по категории "Заказ"')
plt.legend()
plt.show()

## Прогноз "Информация по сети"

In [ ]:
start_date = pd.to_datetime('2023-05-01')
beg_date = pd.to_datetime('2022-07-01')
filtered_d = cal[cal['category'] == 'Информация по сети']
filtered_d1 = filtered_d[(filtered_d['call_dt'] < start_date) & (filtered_d['call_dt'] > beg_date)]
filtered_d1

In [ ]:
model = COPOD(contamination=0.05)
filtered_data1 = filtered_d1
model.fit(filtered_data1['num_calls'].values.reshape(-1, 1))
outliers = model.predict(filtered_data1['num_calls'].values.reshape(-1, 1))
count = np.count_nonzero(outliers == 1)     
filtered_data1['outliers'] = outliers
filtered_data1 = filtered_data1.reset_index(drop=True)

outl = filtered_data1.loc[filtered_data1['outliers'] == 1]
outl

In [ ]:
plt.figure(figsize=(50,10))
plt.plot(filtered_data1['call_dt'], filtered_data1['num_calls'], label = 'звонки')
plt.plot(outl['call_dt'], outl['num_calls'], marker = 'o', linestyle = '', label = 'звонки')

In [ ]:
# Замена значений выбросов в столбце calls
for i in range(len(filtered_data1)):
    if filtered_data1.at[i, 'outliers'] == 1:
        prev_mean = filtered_data1.at[max(i-2, 0), 'num_calls']
        filtered_data1.at[i, 'num_calls'] = (prev_mean) 

filtered_d1 = filtered_data1       
plt.figure(figsize=(50,10))
plt.plot(filtered_data1['call_dt'], filtered_data1['num_calls'], label = 'звонки')

In [ ]:
result = adfuller(filtered_d1['num_calls'])
result

In [ ]:
model = SimpleExpSmoothing(filtered_d1['num_calls'])
forecast_inf_store = pd.DataFrame(columns=["call_dt", "forc"], dtype=float)
forecast = model.fit().forecast(steps=120)
for i, value in enumerate(forecast):
    date = start_date + pd.DateOffset(days=i)
    forecast_inf_store.loc[i] = [date, value]
forecast_inf_store

In [ ]:
plt.figure(figsize=(16, 6))
plt.plot(filtered_d['call_dt'], filtered_d['num_calls'], label='Actual')
plt.plot(forecast_inf_store['call_dt'], forecast_inf_store['forc'], label='Forecast')
plt.xlabel('Дата')
plt.ylabel('Количество звонков')
plt.title('Прогноз по категории "Информация по сети"')
plt.legend()
plt.show()

## Прогноз "Звонок не состоялся"

In [ ]:
start_date = pd.to_datetime('2023-05-01')
beg_date = pd.to_datetime('2022-04-01')
filtered_d = cal[cal['category'] == 'Звонок не состоялся']
filtered_d1 = filtered_d[(filtered_d['call_dt'] < start_date) & (filtered_d['call_dt'] > beg_date)]
filtered_d1

In [ ]:
model = COPOD(contamination=0.05)
filtered_data1 = filtered_d1
model.fit(filtered_data1['num_calls'].values.reshape(-1, 1))
outliers = model.predict(filtered_data1['num_calls'].values.reshape(-1, 1))
count = np.count_nonzero(outliers == 1)     
filtered_data1['outliers'] = outliers
filtered_data1 = filtered_data1.reset_index(drop=True)

outl = filtered_data1.loc[filtered_data1['outliers'] == 1]
outl

In [ ]:
plt.figure(figsize=(50,10))
plt.plot(filtered_data1['call_dt'], filtered_data1['num_calls'], label = 'звонки')
plt.plot(outl['call_dt'], outl['num_calls'], marker = 'o', linestyle = '', label = 'звонки')

In [ ]:
# Замена значений выбросов в столбце calls
for i in range(len(filtered_data1)):
    if filtered_data1.at[i, 'outliers'] == 1:
        prev_mean = filtered_data1.at[max(i-2, 0), 'num_calls']
        filtered_data1.at[i, 'num_calls'] = (prev_mean) 

filtered_d1 = filtered_data1       
plt.figure(figsize=(50,10))
plt.plot(filtered_data1['call_dt'], filtered_data1['num_calls'], label = 'звонки')

In [ ]:
result = adfuller(filtered_d1['num_calls'])
result

In [ ]:
model = SimpleExpSmoothing(filtered_d1['num_calls'])
forecast_call_failed = pd.DataFrame(columns=["call_dt", "forc"], dtype=float)
forecast = model.fit().forecast(steps=120)
for i, value in enumerate(forecast):
    date = start_date + pd.DateOffset(days=i)
    forecast_call_failed.loc[i] = [date, value]
forecast_call_failed

In [ ]:
plt.figure(figsize=(16, 6))
plt.plot(filtered_d['call_dt'], filtered_d['num_calls'], label='Actual')
plt.plot(forecast_call_failed['call_dt'], forecast_call_failed['forc'], label='Forecast')
plt.xlabel('Дата')
plt.ylabel('Количество звонков')
plt.title('Прогноз по категории "Звонок не состоялся"')
plt.legend()
plt.show()

## Прогноз "Другое"

In [ ]:
start_date = pd.to_datetime('2023-05-01')
beg_date = pd.to_datetime('2021-07-30')
filtered_d = cal[cal['category'] == 'Другое']
filtered_d1 = filtered_d[(filtered_d['call_dt'] < start_date) & (filtered_d['call_dt'] > beg_date)]
filtered_d1

In [ ]:
model = COPOD(contamination=0.05)
filtered_data1 = filtered_d1
model.fit(filtered_data1['num_calls'].values.reshape(-1, 1))
outliers = model.predict(filtered_data1['num_calls'].values.reshape(-1, 1))
count = np.count_nonzero(outliers == 1)     
filtered_data1['outliers'] = outliers
filtered_data1 = filtered_data1.reset_index(drop=True)

outl = filtered_data1.loc[filtered_data1['outliers'] == 1]
outl

In [ ]:
plt.figure(figsize=(50,10))
plt.plot(filtered_data1['call_dt'], filtered_data1['num_calls'], label = 'звонки')
plt.plot(outl['call_dt'], outl['num_calls'], marker = 'o', linestyle = '', label = 'звонки')

In [ ]:
# Замена значений выбросов в столбце calls
for i in range(len(filtered_data1)):
    if filtered_data1.at[i, 'outliers'] == 1:
        prev_mean = filtered_data1.at[max(i-2, 0), 'num_calls']
        filtered_data1.at[i, 'num_calls'] = (prev_mean) 

filtered_d1 = filtered_data1       
plt.figure(figsize=(50,10))
plt.plot(filtered_data1['call_dt'], filtered_data1['num_calls'], label = 'звонки')

In [ ]:
result = adfuller(filtered_d1['num_calls'])
result

In [ ]:
model = SimpleExpSmoothing(filtered_d1['num_calls'])
forecast_other = pd.DataFrame(columns=["call_dt", "forc"], dtype=float)
forecast = model.fit().forecast(steps=120)
for i, value in enumerate(forecast):
    date = start_date + pd.DateOffset(days=i)
    forecast_other.loc[i] = [date, value]
forecast_other

In [ ]:
plt.figure(figsize=(16, 6))
plt.plot(filtered_d['call_dt'], filtered_d['num_calls'], label='Actual')
plt.plot(forecast_other['call_dt'], forecast_other['forc'], label='Forecast')
plt.xlabel('Дата')
plt.ylabel('Количество звонков')
plt.title('Прогноз по категории "Другое"')
plt.legend()
plt.show()

## Прогноз "Интернет-магазин"

In [ ]:
start_date = pd.to_datetime('2023-05-01')
beg_date = pd.to_datetime('2022-03-01')
filtered_d = cal[cal['category'] == 'Интернет-магазин']
filtered_d1 = filtered_d[(filtered_d['call_dt'] < start_date) & (filtered_d['call_dt'] > beg_date)]
filtered_d1

In [ ]:
model = COPOD(contamination=0.05)
filtered_data1 = filtered_d1
model.fit(filtered_data1['num_calls'].values.reshape(-1, 1))
outliers = model.predict(filtered_data1['num_calls'].values.reshape(-1, 1))
count = np.count_nonzero(outliers == 1)     
filtered_data1['outliers'] = outliers
filtered_data1 = filtered_data1.reset_index(drop=True)

outl = filtered_data1.loc[filtered_data1['outliers'] == 1]
outl

In [ ]:
plt.figure(figsize=(50,10))
plt.plot(filtered_data1['call_dt'], filtered_data1['num_calls'], label = 'звонки')
plt.plot(outl['call_dt'], outl['num_calls'], marker = 'o', linestyle = '', label = 'звонки')

In [ ]:
# Замена значений выбросов в столбце calls
for i in range(len(filtered_data1)):
    if filtered_data1.at[i, 'outliers'] == 1:
        prev_mean = filtered_data1.at[max(i-2, 0), 'num_calls']
        filtered_data1.at[i, 'num_calls'] = (prev_mean) 

filtered_d1 = filtered_data1       
plt.figure(figsize=(50,10))
plt.plot(filtered_data1['call_dt'], filtered_data1['num_calls'], label = 'звонки')

In [ ]:
result = adfuller(filtered_d1['num_calls'])
result

In [ ]:
model = SimpleExpSmoothing(filtered_d1['num_calls'])
forecast_online_store = pd.DataFrame(columns=["call_dt", "forc"], dtype=float)
forecast = model.fit().forecast(steps=120)
for i, value in enumerate(forecast):
    date = start_date + pd.DateOffset(days=i)
    forecast_online_store.loc[i] = [date, value]
forecast_online_store

In [ ]:
plt.figure(figsize=(16, 6))
plt.plot(filtered_d['call_dt'], filtered_d['num_calls'], label='Actual')
plt.plot(forecast_online_store['call_dt'], forecast_online_store['forc'], label='Forecast')
plt.xlabel('Дата')
plt.ylabel('Количество звонков')
plt.title('Прогноз по категории "Интернет-магазин"')
plt.legend()
plt.show()

## Прогноз "Информация о товаре"

In [ ]:
start_date = pd.to_datetime('2023-05-01')
beg_date = pd.to_datetime('2022-04-30')
filtered_d = cal[cal['category'] == 'Информация о товаре']
filtered_d1 = filtered_d[(filtered_d['call_dt'] < start_date) & (filtered_d['call_dt'] > beg_date)]
filtered_d1

In [ ]:
model = COPOD(contamination=0.05)
filtered_data1 = filtered_d1
model.fit(filtered_data1['num_calls'].values.reshape(-1, 1))
outliers = model.predict(filtered_data1['num_calls'].values.reshape(-1, 1))
count = np.count_nonzero(outliers == 1)     
filtered_data1['outliers'] = outliers
filtered_data1 = filtered_data1.reset_index(drop=True)

outl = filtered_data1.loc[filtered_data1['outliers'] == 1]
outl

In [ ]:
plt.figure(figsize=(50,10))
plt.plot(filtered_data1['call_dt'], filtered_data1['num_calls'], label = 'звонки')
plt.plot(outl['call_dt'], outl['num_calls'], marker = 'o', linestyle = '', label = 'звонки')

In [ ]:
# Замена значений выбросов в столбце calls
for i in range(len(filtered_data1)):
    if filtered_data1.at[i, 'outliers'] == 1:
        prev_mean = filtered_data1.at[max(i-2, 0), 'num_calls']
        filtered_data1.at[i, 'num_calls'] = (prev_mean) 

filtered_d1 = filtered_data1       
plt.figure(figsize=(50,10))
plt.plot(filtered_data1['call_dt'], filtered_data1['num_calls'], label = 'звонки')

In [ ]:
result = adfuller(filtered_d1['num_calls'])
result

In [ ]:
model = SimpleExpSmoothing(filtered_d1['num_calls'])
forecast_goods = pd.DataFrame(columns=["call_dt", "forc"], dtype=float)
forecast = model.fit().forecast(steps=120)
for i, value in enumerate(forecast):
    date = start_date + pd.DateOffset(days=i)
    forecast_goods.loc[i] = [date, value]
forecast_goods

In [ ]:
plt.figure(figsize=(16, 6))
plt.plot(filtered_d['call_dt'], filtered_d['num_calls'], label='Actual')
plt.plot(forecast_goods['call_dt'], forecast_goods['forc'], label='Forecast')
plt.xlabel('Дата')
plt.ylabel('Сумма звонков')
plt.title('Прогноз по категории "Информация о товаре"')
plt.legend()
plt.show()

## Прогноз "Выгода с Улыбкой"

In [ ]:
# обучающая выборка
filtered_data = cal[cal['category'] == 'Выгода с Улыбкой']
filtered_data = filtered_data[filtered_data['call_dt'] > '2021-01-01']
filtered_data = filtered_data.reset_index(drop=True)
filtered_data

In [ ]:
model = COPOD(contamination=0.025)
filtered_data1 = filtered_data
model.fit(filtered_data1['num_calls'].values.reshape(-1, 1))
outliers = model.predict(filtered_data1['num_calls'].values.reshape(-1, 1))
count = np.count_nonzero(outliers == 1)     
filtered_data1['outliers'] = outliers
filtered_data1 = filtered_data1.reset_index(drop=True)

In [ ]:
# Построение графика
outl = filtered_data1.loc[filtered_data1['outliers'] == 1]
plt.figure(figsize=(50,10))
plt.plot(filtered_data1['call_dt'], filtered_data1['num_calls'], label = 'звонки')
plt.plot(outl['call_dt'], outl['num_calls'], marker = 'o', linestyle = '', label = 'звонки')

In [ ]:
# Замена значений выбросов в столбце calls
for i in range(len(filtered_data1)):
    if filtered_data1.at[i, 'outliers'] == 1:
        prev_mean = filtered_data1.at[max(i-2, 0), 'num_calls']
        filtered_data1.at[i, 'num_calls'] = (prev_mean) 

plt.figure(figsize=(50,10))
plt.plot(filtered_data1['call_dt'], filtered_data1['num_calls'], label = 'звонки')

In [ ]:
model = Prophet(seasonality_mode='additive')
model.add_seasonality(name='yearly', period=365, fourier_order=10)
model.add_seasonality(name='monthly', period=30, fourier_order=10)
model.add_seasonality(name='weekly', period=7, fourier_order=10)
y = filtered_data[['call_dt','num_calls','actiya']].iloc[:-90]
y = y.rename(columns={'call_dt': 'ds', 'num_calls': 'y'})
model.fit(y)
forecast_ds = model.make_future_dataframe(periods=120)
forecast_benefit = model.predict(forecast_ds)
forecast_benefit.tail()

plt.figure(figsize=(16, 6))
plt.plot(filtered_data['call_dt'], filtered_data['num_calls'], label='Actual')
plt.plot(forecast_benefit['ds'][-120:], forecast_benefit['yhat'][-120:], label='Forecast')
plt.xlabel('Дата')
plt.ylabel('Количество звонков')
plt.title('Прогноз по категории "Акции"')
plt.legend()
plt.show()

In [ ]:
def smape(y_true, y_pred):
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

print ("Ошибка модели:", smape(filtered_data["num_calls"][-212:], forecast["yhat"][-212:]))

In [ ]:
forecast_benefit = forecast_benefit.pivot_table(index = 'ds',values = 'yhat').reset_index()
forecast_benefit = forecast_benefit.rename(columns={'ds':'call_dt', 'yhat':'benefit'})
forecast_benefit

## Прогноз "Вакансия"

In [ ]:
# обучающая выборка
filtered_data = cal[cal['category'] == 'Вакансия']
filtered_data = filtered_data[filtered_data['call_dt'] > '2023-01-01']
filtered_data = filtered_data.reset_index(drop=True)
filtered_data

In [ ]:
model = COPOD(contamination=0.025)
filtered_data1 = filtered_data
model.fit(filtered_data1['num_calls'].values.reshape(-1, 1))
outliers = model.predict(filtered_data1['num_calls'].values.reshape(-1, 1))
count = np.count_nonzero(outliers == 1)     
filtered_data1['outliers'] = outliers
filtered_data1 = filtered_data1.reset_index(drop=True)

In [ ]:
# Построение графика
outl = filtered_data1.loc[filtered_data1['outliers'] == 1]
plt.figure(figsize=(50,10))
plt.plot(filtered_data1['call_dt'], filtered_data1['num_calls'], label = 'звонки')
plt.plot(outl['call_dt'], outl['num_calls'], marker = 'o', linestyle = '', label = 'звонки')

In [ ]:
# Замена значений выбросов в столбце calls
for i in range(len(filtered_data1)):
    if filtered_data1.at[i, 'outliers'] == 1:
        prev_mean = filtered_data1.at[max(i-2, 0), 'num_calls']
        filtered_data1.at[i, 'num_calls'] = (prev_mean) 

plt.figure(figsize=(50,10))
plt.plot(filtered_data1['call_dt'], filtered_data1['num_calls'], label = 'звонки')

In [ ]:
model = Prophet(seasonality_mode='multiplicative')
model.add_seasonality(name='yearly', period=365, fourier_order=10)
#model.add_seasonality(name='monthly', period=30, fourier_order=10)
model.add_seasonality(name='weekly', period=7, fourier_order=10)
y = filtered_data[['call_dt','num_calls','actiya']].iloc[:-105]
y = y.rename(columns={'call_dt': 'ds', 'num_calls': 'y'})
model.fit(y)
forecast_ds = model.make_future_dataframe(periods=120)
forecast_vacancy = model.predict(forecast_ds)
forecast_vacancy.tail()

plt.figure(figsize=(16, 6))
plt.plot(filtered_data['call_dt'], filtered_data['num_calls'], label='Actual')
plt.plot(forecast_vacancy['ds'][-120:], forecast_vacancy['yhat'][-120:], label='Forecast')
plt.xlabel('Дата')
plt.ylabel('Количество звонков')
plt.title('Прогноз по категории "Вакансии"')
plt.legend()
plt.show()

In [ ]:
def smape(y_true, y_pred):
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

print ("Ошибка модели:", smape(filtered_data["num_calls"][-212:], forecast_vacancy["yhat"][-212:]))

In [ ]:
forecast_vacancy = forecast_vacancy.pivot_table(index = 'ds',values = 'yhat').reset_index()
forecast_vacancy = forecast_vacancy.rename(columns={'ds':'call_dt', 'yhat':'vacancy'})
forecast_vacancy

## Прогноз "Сотрудники"

In [ ]:
# обучающая выборка
filtered_data = cal[cal['category'] == 'Сотрудники']
filtered_data = filtered_data[filtered_data['call_dt'] > '2022-07-01']
filtered_data = filtered_data.reset_index(drop=True)
filtered_data

In [ ]:
model = COPOD(contamination=0.025)
filtered_data1 = filtered_data
model.fit(filtered_data1['num_calls'].values.reshape(-1, 1))
outliers = model.predict(filtered_data1['num_calls'].values.reshape(-1, 1))
count = np.count_nonzero(outliers == 1)     
filtered_data1['outliers'] = outliers
filtered_data1 = filtered_data1.reset_index(drop=True)

In [ ]:
# Построение графика
outl = filtered_data1.loc[filtered_data1['outliers'] == 1]
plt.figure(figsize=(50,10))
plt.plot(filtered_data1['call_dt'], filtered_data1['num_calls'], label = 'звонки')
plt.plot(outl['call_dt'], outl['num_calls'], marker = 'o', linestyle = '', label = 'звонки')

In [ ]:
# Замена значений выбросов в столбце calls
for i in range(len(filtered_data1)):
    if filtered_data1.at[i, 'outliers'] == 1:
        prev_mean = filtered_data1.at[max(i-2, 0), 'num_calls']
        filtered_data1.at[i, 'num_calls'] = (prev_mean) 

plt.figure(figsize=(50,10))
plt.plot(filtered_data1['call_dt'], filtered_data1['num_calls'], label = 'звонки')

In [ ]:
model = Prophet(seasonality_mode='additive')
model.add_seasonality(name='yearly', period=365, fourier_order=10)
#model.add_seasonality(name='monthly', period=30, fourier_order=10)
model.add_seasonality(name='weekly', period=7, fourier_order=10)
y = filtered_data[['call_dt','num_calls','actiya']].iloc[:-90]
y = y.rename(columns={'call_dt': 'ds', 'num_calls': 'y'})
model.fit(y)
forecast_ds = model.make_future_dataframe(periods=120)
forecast_staff = model.predict(forecast_ds)
forecast_staff.tail()

plt.figure(figsize=(16, 6))
plt.plot(filtered_data['call_dt'], filtered_data['num_calls'], label='Actual')
plt.plot(forecast_staff['ds'][-120:], forecast_staff['yhat'][-120:], label='Forecast')
plt.xlabel('Дата')
plt.ylabel('Количество звонков')
plt.title('Проноз по категории "Сотрудники"')
plt.legend()
plt.show()

In [ ]:
def smape(y_true, y_pred):
    return 100/len(y_true) * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))

print ("Ошибка модели:", smape(filtered_data["num_calls"][-212:], forecast_staff["yhat"][-212:]))

In [ ]:
forecast_staff = forecast_staff.pivot_table(index = 'ds',values = 'yhat').reset_index()
forecast_staff = forecast_staff.rename(columns={'ds':'call_dt', 'yhat':'staff'})
forecast_staff

##       

## Формирование таблицы с прогнозами по категориям

In [ ]:
merged_df = pd.merge(forecast_complaints, forecast_marketplaces, on='call_dt')
merged_1 = pd.merge(merged_df, forecast_order, on='call_dt')
merged_2 = pd.merge(merged_1, forecast_inf_store, on='call_dt')
merged_3 = pd.merge(merged_2, forecast_call_failed, on='call_dt')
merged_4 = pd.merge(merged_3, forecast_other, on='call_dt')
merged_5 = pd.merge(merged_4, forecast_online_store, on='call_dt')
merged_6 = pd.merge(merged_5, forecast_goods, on='call_dt')

In [ ]:
merged_6.set_axis(['call_dt', 'complaints', 'marketplaces', 'order', 'inf_store', 'call_failed', 'other', 'online_store', 'goods'], axis=1, inplace=True)
merged_7 = pd.merge(merged_6, forecast_staff, on='call_dt')
merged_8 = pd.merge(merged_7, forecast_benefit, on='call_dt')
forecast_table = pd.merge(merged_8, forecast_vacancy, on='call_dt')
forecast_table['sum'] = forecast_table['complaints'] + forecast_table['marketplaces'] + forecast_table['order'] + forecast_table['inf_store'] +forecast_table['call_failed'] + forecast_table['other'] + forecast_table['online_store'] + forecast_table['goods']+ forecast_table['staff']+ forecast_table['benefit']+ forecast_table['vacancy']

In [ ]:
forecast_table

In [ ]:
forecast_line = forecast_table.pivot_table(index='call_dt', values='sum').reset_index()


plt.figure(figsize=(50,10))
plt.plot(forecast_line['call_dt'], forecast_line['sum'], label = 'Прогноз звонков')
plt.plot(all_calls['call_dt'], all_calls['sum_calls'], label = 'Прогноз звонков')